## IMPORTING MODULES

In [1]:
import os, glob
import numpy as np
import pandas as pd
from numba import njit
import math

import matplotlib
matplotlib.use("Agg")   # Kaggle-safe backend
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as mpl_Polygon


## Santa Tree Shape Definition

In [2]:
PX = np.array(
    [0.0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075,
     -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125],
    dtype=np.float64
)

PY = np.array(
    [0.8, 0.5, 0.5, 0.25, 0.25, 0.0, 0.0, -0.2,
     -0.2, 0.0, 0.0, 0.25, 0.25, 0.5, 0.5],
    dtype=np.float64
)


## Geometry + Collision Functions (Numba)

In [3]:
@njit
def get_tree_poly(cx, cy, deg):
    a = deg * 0.017453292519943295
    ca, sa = math.cos(a), math.sin(a)
    res = np.empty((15, 2), dtype=np.float64)
    for i in range(15):
        res[i, 0] = (ca * PX[i] - sa * PY[i]) + cx
        res[i, 1] = (sa * PX[i] + ca * PY[i]) + cy
    return res


@njit
def is_overlapping(p1, p2):
    m1x, M1x = np.min(p1[:,0]), np.max(p1[:,0])
    m1y, M1y = np.min(p1[:,1]), np.max(p1[:,1])
    m2x, M2x = np.min(p2[:,0]), np.max(p2[:,0])
    m2y, M2y = np.min(p2[:,1]), np.max(p2[:,1])

    if M1x < m2x or M2x < m1x or M1y < m2y or M2y < m1y:
        return False

    for poly in (p1, p2):
        for i in range(15):
            p1_pt = poly[i]
            p2_pt = poly[(i + 1) % 15]
            ax, ay = -(p2_pt[1] - p1_pt[1]), (p2_pt[0] - p1_pt[0])

            min1, max1 = 1e30, -1e30
            for k in range(15):
                d = p1[k,0]*ax + p1[k,1]*ay
                min1 = min(min1, d)
                max1 = max(max1, d)

            min2, max2 = 1e30, -1e30
            for k in range(15):
                d = p2[k,0]*ax + p2[k,1]*ay
                min2 = min(min2, d)
                max2 = max(max2, d)

            if max1 < min2 or max2 < min1:
                return False

    return True


## Bounding Box + Optimizer

In [4]:
@njit
def get_side_fast(xs, ys, ds, n):
    mnx, mxx, mny, mxy = 1e30, -1e30, 1e30, -1e30
    for i in range(n):
        a = ds[i] * 0.017453292519943295
        ca, sa = math.cos(a), math.sin(a)
        for j in range(15):
            tx = (ca * PX[j] - sa * PY[j]) + xs[i]
            ty = (sa * PX[j] + ca * PY[j]) + ys[i]
            mnx = min(mnx, tx)
            mxx = max(mxx, tx)
            mny = min(mny, ty)
            mxy = max(mxy, ty)
    return max(mxx - mnx, mxy - mny)


@njit
def aggressive_optimize(xs, ys, ds, n):
    curr_side = get_side_fast(xs, ys, ds, n)
    polys = [get_tree_poly(xs[i], ys[i], ds[i]) for i in range(n)]

    moves = np.array([0.05, 0.01, 0.002, -0.05, -0.01, -0.002])
    rots = np.array([0.0, 90.0, 180.0, 270.0, 1.0, -1.0])

    for _ in range(2):
        improved = False
        for i in range(n):
            ox, oy, od = xs[i], ys[i], ds[i]
            for dx in moves:
                for dy in moves:
                    for dr in rots:
                        nx, ny, nd = ox+dx, oy+dy, (od+dr)%360
                        npoly = get_tree_poly(nx, ny, nd)

                        bad = False
                        for j in range(n):
                            if i!=j and is_overlapping(npoly, polys[j]):
                                bad = True
                                break

                        if not bad:
                            xs[i], ys[i], ds[i] = nx, ny, nd
                            s = get_side_fast(xs, ys, ds, n)
                            if s < curr_side:
                                curr_side = s
                                polys[i] = npoly
                                improved = True
                            else:
                                xs[i], ys[i], ds[i] = ox, oy, od
        if not improved:
            break


## Load Best Inputs

In [5]:
def load_all_best():
    configs = {}
    for n in range(1, 201):
        xs = np.zeros(n)
        ys = np.zeros(n)
        ds = np.zeros(n)
        for i in range(n):
            xs[i] = i % int(np.sqrt(n))
            ys[i] = i // int(np.sqrt(n))
            ds[i] = 0.0
        configs[n] = pd.DataFrame({"x": xs, "y": ys, "deg": ds})
    return configs


## Main Execution + Submission

In [6]:
configs = load_all_best()

if not configs:
    raise RuntimeError("No valid CSV solutions found. Check dataset path.")

final_x = np.zeros(20100, dtype=np.float64)
final_y = np.zeros(20100, dtype=np.float64)
final_d = np.zeros(20100, dtype=np.float64)

max_n = max(configs.keys())

for n in range(max_n, 0, -1):

    if n not in configs:
        continue

    idx = n * (n - 1) // 2
    cur = configs[n]

    x = cur.x.values
    y = cur.y.values
    d = cur.deg.values

    aggressive_optimize(x, y, d, n)

    final_x[idx:idx+n] = x
    final_y[idx:idx+n] = y
    final_d[idx:idx+n] = d

res = []
for n in range(1, max_n + 1):
    idx = n * (n - 1) // 2
    for i in range(n):
        res.append({
            "id": f"{n:03d}_{i}",
            "x": f"s{final_x[idx+i]:.16f}",
            "y": f"s{final_y[idx+i]:.16f}",
            "deg": f"s{(final_d[idx+i] % 360):.16f}"
        })

pd.DataFrame(res).to_csv("submission.csv", index=False)
